In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt

import numpy as np
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
import torchvision
from torchvision import datasets, transforms, models
from PIL import Image
from torch.autograd import Variable


In [ ]:
data_dir = "./data/actual"
input_size = 224
num_classes = 24

# Batch size for training (change depending on how much memory you have)
batch_size = 1
# Data augmentation and normalization for training
# Just normalization for validation
test_transforms = transforms.Compose([
        transforms.Resize(input_size),      
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
image_dataset = datasets.ImageFolder(data_dir, test_transforms)
# Create training and validation dataloaders
trainloader = torch.utils.data.DataLoader(image_dataset, batch_size=batch_size, shuffle=True, num_workers=4)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = torch.load('/home/jasayles/Comp-Vis-Tour/model.pth')
model.eval()
print("yes")

In [ ]:
def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()


In [ ]:
classes = ['Byrum', 'Decker','Dunn','Eflame',
           'FA','Flag','Hard','Hart','Helio',
           'Kard','Libr','Mart','Morr','Myer',
           'OLT','Passag','Peace','Prayer','Rear','Rice','Rock','Seal','Smith','Statue']
dataiter = iter(trainloader)
images, labels = dataiter.next()
print(labels)
# print images
imshow(torchvision.utils.make_grid(images))
print('GroundTruth: ', ' '.join('%1s' % classes[labels[j]] for j in range(1)))
outputs = model(images.to(device))
_, predicted = torch.max(outputs, 1)

print('Predicted: ', ' '.join('%1s' % classes[predicted[j]]
                              for j in range(1)))

In [ ]:

test_transform = transforms.Compose([transforms.Resize(input_size),      
                                     transforms.CenterCrop(input_size),
                                     transforms.ToTensor(),
                                      #transforms.Normalize([0.485, 0.456, 0.406],
                                      #                     [0.229, 0.224, 0.225])
                                     ])



In [ ]:
def predict_image(image):
    image_tensor = test_transform(image).float()
    image_tensor = image_tensor.unsqueeze_(0)
    input = Variable(image_tensor)
    input = input.to(device)
    output = model(input)
    index = output.data.cpu().numpy().argmax()
    return index  

def get_random_images(num):
    data = datasets.ImageFolder(data_dir, transform=test_transform)
    classes = data.classes
    indices = list(range(len(data)))
    np.random.shuffle(indices)
    idx = indices[:num]
    from torch.utils.data.sampler import SubsetRandomSampler
    sampler = SubsetRandomSampler(idx)
    loader = torch.utils.data.DataLoader(data, sampler=sampler, batch_size=num)
    dataiter = iter(loader)
    images, labels = dataiter.next()
    return images, labels

In [ ]:
to_pil = transforms.ToPILImage()
images, labels = get_random_images(1)
fig=plt.figure(figsize=(10,10))
for ii in range(len(images)):
    image = to_pil(images[ii])
    index = predict_image(image)
    sub = fig.add_subplot(1, len(images), ii+1)
    res = int(labels[ii]) == index
    sub.set_title(str(classes[index]) + ":" + str(res))
    plt.axis('off')
    plt.imshow(image)
plt.show()

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torchvision
from torchvision import datasets, transforms, models
from torch.autograd import Variable

data_dir = "./data/actual"
input_size = 224
num_classes = 24

batch_size = 1
test_transforms = transforms.Compose([transforms.Resize(input_size),      
                                     transforms.CenterCrop(input_size),
                                     transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])
                                     ])
image_dataset = datasets.ImageFolder(data_dir, test_transforms)
trainloader = torch.utils.data.DataLoader(image_dataset, batch_size=batch_size, shuffle=True, num_workers=4)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = torch.load('/home/jasayles/Comp-Vis-Tour/model.pth')
model.eval()

classes = ['Byrum Hall', 'Decker Hall','Dunn Hall','Eternal Flame',
           'Fine Arts','International Plaza','Hardacre Hall','Hartung Hall','Helios',
           'Kardatzke Wellnes Center','Nicholson Library','Martin Hall','Morrison Hall',
           'Myers Hall', 'OLT Student Center','Passages','Peace Pole','Centennial Prayer Labyrinth',
           'Reardon Auditorium','Rice Hall','Pioneer Rock','Campus Seal','Smith Hall','Morrison Statue']

dataiter = iter(trainloader)
images, labels = dataiter.next()

outputs = model(images.to(device))
_, predicted = torch.max(outputs, 1)

print('Predicted class is {}:'.format(classes[predicted[0]]))